In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1587089,2021-08-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1587090,2021-08-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1587091,2021-08-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1587092,2021-08-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
41607,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
41609,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
41611,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
41613,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
41615,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
969060,2021-08-23,Arkansas,Arkansas,5001,2972,49.00,5000,Arkansas,AR,Arkansas,State,3017804
969062,2021-08-24,Arkansas,Arkansas,5001,2992,50.00,5000,Arkansas,AR,Arkansas,State,3017804
969064,2021-08-25,Arkansas,Arkansas,5001,3020,50.00,5000,Arkansas,AR,Arkansas,State,3017804
969066,2021-08-26,Arkansas,Arkansas,5001,3034,51.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1587089,2021-08-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1587090,2021-08-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1587091,2021-08-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1587092,2021-08-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-08-27') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
584,2021-08-27,Snohomish,Washington,50079,646.00,53061,WA,County,822083,6091.72,78.58
1166,2021-08-27,Cook,Illinois,588099,11288.00,17031,IL,County,5150233,11418.88,219.17
1747,2021-08-27,Orange,California,303381,5202.00,6059,CA,County,3175692,9553.22,163.81
2327,2021-08-27,Maricopa,Arizona,633745,10748.00,4013,AZ,County,4485414,14129.02,239.62
2907,2021-08-27,Los Angeles,California,1397943,25211.00,6037,CA,County,10039107,13924.97,251.13
...,...,...,...,...,...,...,...,...,...,...,...
1585941,2021-08-27,Wheeler,Oregon,60,1.00,41069,OR,County,1332,4504.50,75.08
1586260,2021-08-27,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1586548,2021-08-27,Esmeralda,Nevada,42,0.00,32009,NV,County,873,4811.00,0.00
1586832,2021-08-27,Loving,Texas,6,0.00,48301,TX,County,169,3550.30,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
584,2021-08-27,Snohomish,Washington,50079,646.00,53061,WA,County,822083,6091.72,78.58,78.58,6091.72
1166,2021-08-27,Cook,Illinois,588099,11288.00,17031,IL,County,5150233,11418.88,219.17,219.17,11418.88
1747,2021-08-27,Orange,California,303381,5202.00,6059,CA,County,3175692,9553.22,163.81,163.81,9553.22
2327,2021-08-27,Maricopa,Arizona,633745,10748.00,4013,AZ,County,4485414,14129.02,239.62,239.62,14129.02
2907,2021-08-27,Los Angeles,California,1397943,25211.00,6037,CA,County,10039107,13924.97,251.13,251.13,13924.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585941,2021-08-27,Wheeler,Oregon,60,1.00,41069,OR,County,1332,4504.50,75.08,75.08,4504.50
1586260,2021-08-27,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1586548,2021-08-27,Esmeralda,Nevada,42,0.00,32009,NV,County,873,4811.00,0.00,0.00,4811.00
1586832,2021-08-27,Loving,Texas,6,0.00,48301,TX,County,169,3550.30,0.00,0.00,3550.30


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1013527,2021-08-27,Hale,Alabama,2608,80.00,1065,AL,County,14651,17800.83,546.04,546.04,17800.83,1
1103747,2021-08-27,Clarke,Alabama,4126,68.00,1025,AL,County,23622,17466.77,287.87,287.87,17466.77,2
701954,2021-08-27,Etowah,Alabama,16849,387.00,1055,AL,County,102268,16475.34,378.42,378.42,16475.34,3
588606,2021-08-27,Franklin,Alabama,5122,89.00,1059,AL,County,31362,16331.87,283.78,283.78,16331.87,4
792081,2021-08-27,Lowndes,Alabama,1578,55.00,1085,AL,County,9726,16224.55,565.49,565.49,16224.55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966999,2021-08-27,Converse,Wyoming,1388,22.00,56009,WY,County,13822,10041.96,159.17,159.17,10041.96,19
967517,2021-08-27,Sublette,Wyoming,915,10.00,56035,WY,County,9831,9307.29,101.72,101.72,9307.29,20
1203784,2021-08-27,Lincoln,Wyoming,1747,14.00,56023,WY,County,19830,8809.88,70.60,70.60,8809.88,21
1341030,2021-08-27,Niobrara,Wyoming,205,3.00,56027,WY,County,2356,8701.19,127.33,127.33,8701.19,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
792081,2021-08-27,Lowndes,Alabama,1578,55.00,1085,AL,County,9726,16224.55,565.49,565.49,16224.55,5,1
1013527,2021-08-27,Hale,Alabama,2608,80.00,1065,AL,County,14651,17800.83,546.04,546.04,17800.83,1,2
854478,2021-08-27,Greene,Alabama,1113,39.00,1063,AL,County,8111,13722.11,480.83,480.83,13722.11,35,3
335801,2021-08-27,Walker,Alabama,8915,296.00,1127,AL,County,63521,14034.73,465.99,465.99,14034.73,33,4
699870,2021-08-27,Clay,Alabama,1909,60.00,1027,AL,County,13235,14423.88,453.34,453.34,14423.88,26,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509012,2021-08-27,Weston,Wyoming,773,6.00,56045,WY,County,6927,11159.23,86.62,86.62,11159.23,14,19
1203784,2021-08-27,Lincoln,Wyoming,1747,14.00,56023,WY,County,19830,8809.88,70.60,70.60,8809.88,21,20
1173576,2021-08-27,Uinta,Wyoming,2914,14.00,56041,WY,County,20226,14407.20,69.22,69.22,14407.20,3,21
788961,2021-08-27,Albany,Wyoming,5147,19.00,56001,WY,County,38880,13238.17,48.87,48.87,13238.17,5,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,78.58,6091.72,21,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,78.58,6091.72,21,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,78.58,6091.72,21,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,78.58,6091.72,21,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,78.58,6091.72,21,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583400,2021-08-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1583401,2021-08-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1583402,2021-08-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1583403,2021-08-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
1009323,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,546.04,17800.83,2,1,1.00
1009324,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,546.04,17800.83,2,1,0.00
1009325,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,546.04,17800.83,2,1,0.00
1009326,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,546.04,17800.83,2,1,0.00
1009327,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,546.04,17800.83,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224108,2021-08-23,Crook,Wyoming,520,13.00,56011,WY,County,7584,6856.54,171.41,171.41,7054.32,8,23,12.00
1224109,2021-08-24,Crook,Wyoming,520,13.00,56011,WY,County,7584,6856.54,171.41,171.41,7054.32,8,23,0.00
1224110,2021-08-25,Crook,Wyoming,527,13.00,56011,WY,County,7584,6948.84,171.41,171.41,7054.32,8,23,7.00
1224111,2021-08-26,Crook,Wyoming,531,13.00,56011,WY,County,7584,7001.58,171.41,171.41,7054.32,8,23,4.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
788909,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16224.55,1,5,1.00,0.00
788910,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16224.55,1,5,0.00,0.00
788911,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16224.55,1,5,0.00,0.00
788912,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16224.55,1,5,0.00,0.00
788913,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,16224.55,1,5,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332084,2021-08-23,Teton,Wyoming,4252,11.00,56039,WY,County,23464,18121.38,46.88,46.88,18564.61,23,1,73.00,0.00
332085,2021-08-24,Teton,Wyoming,4265,11.00,56039,WY,County,23464,18176.78,46.88,46.88,18564.61,23,1,13.00,0.00
332086,2021-08-25,Teton,Wyoming,4300,11.00,56039,WY,County,23464,18325.95,46.88,46.88,18564.61,23,1,35.00,0.00
332087,2021-08-26,Teton,Wyoming,4329,11.00,56039,WY,County,23464,18449.54,46.88,46.88,18564.61,23,1,29.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-08-27') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
401117,2021-08-27,Imperial,California,33128,747.00,6025,CA,County,181215,18281.05,412.22,412.22,18281.05,1,2,36.00,0.00
2907,2021-08-27,Los Angeles,California,1397943,25211.00,6037,CA,County,10039107,13924.97,251.13,251.13,13924.97,2,6,2751.00,30.00
199261,2021-08-27,San Bernardino,California,332238,5339.00,6071,CA,County,2180085,15239.68,244.90,244.90,15239.68,3,4,872.00,2.00
703990,2021-08-27,Inyo,California,1526,39.00,6027,CA,County,18039,8459.45,216.20,216.20,8459.45,4,33,3.00,0.00
88877,2021-08-27,Stanislaus,California,73056,1133.00,6099,CA,County,550660,13266.99,205.75,205.75,13266.99,5,9,479.00,4.00
70655,2021-08-27,San Joaquin,California,89018,1513.00,6077,CA,County,762148,11679.88,198.52,198.52,11679.88,6,16,427.00,3.00
41067,2021-08-27,Riverside,California,340526,4727.00,6065,CA,County,2470546,13783.43,191.33,191.33,13783.43,7,7,938.00,8.00
110274,2021-08-27,Tulare,California,65554,870.00,6107,CA,County,466195,14061.50,186.62,186.62,14061.50,8,5,280.00,3.00
637713,2021-08-27,Merced,California,36352,506.00,6047,CA,County,277680,13091.33,182.22,182.22,13091.33,9,10,239.00,0.00
39989,2021-08-27,Fresno,California,124095,1770.00,6019,CA,County,999101,12420.67,177.16,177.16,12420.67,10,13,470.00,9.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1498426,2021-08-27,Lassen,California,6239,27.00,6035,CA,County,30573,20406.89,88.31,88.31,20406.89,35,1,72.00,0.00
401117,2021-08-27,Imperial,California,33128,747.00,6025,CA,County,181215,18281.05,412.22,412.22,18281.05,1,2,36.00,0.00
854420,2021-08-27,Kings,California,27347,262.00,6031,CA,County,152940,17880.87,171.31,171.31,17880.87,11,3,162.00,0.00
199261,2021-08-27,San Bernardino,California,332238,5339.00,6071,CA,County,2180085,15239.68,244.90,244.90,15239.68,3,4,872.00,2.00
110274,2021-08-27,Tulare,California,65554,870.00,6107,CA,County,466195,14061.50,186.62,186.62,14061.50,8,5,280.00,3.00
2907,2021-08-27,Los Angeles,California,1397943,25211.00,6037,CA,County,10039107,13924.97,251.13,251.13,13924.97,2,6,2751.00,30.00
41067,2021-08-27,Riverside,California,340526,4727.00,6065,CA,County,2470546,13783.43,191.33,191.33,13783.43,7,7,938.00,8.00
250713,2021-08-27,Kern,California,122950,1458.00,6029,CA,County,900202,13658.05,161.96,161.96,13658.05,13,8,564.00,5.00
88877,2021-08-27,Stanislaus,California,73056,1133.00,6099,CA,County,550660,13266.99,205.75,205.75,13266.99,5,9,479.00,4.00
637713,2021-08-27,Merced,California,36352,506.00,6047,CA,County,277680,13091.33,182.22,182.22,13091.33,9,10,239.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
401117,2021-08-27,Imperial,California,33128,747.00,6025,CA,County,181215,18281.05,412.22,412.22,18281.05,1,2,36.00,0.00
2907,2021-08-27,Los Angeles,California,1397943,25211.00,6037,CA,County,10039107,13924.97,251.13,251.13,13924.97,2,6,2751.00,30.00
199261,2021-08-27,San Bernardino,California,332238,5339.00,6071,CA,County,2180085,15239.68,244.90,244.90,15239.68,3,4,872.00,2.00
703990,2021-08-27,Inyo,California,1526,39.00,6027,CA,County,18039,8459.45,216.20,216.20,8459.45,4,33,3.00,0.00
88877,2021-08-27,Stanislaus,California,73056,1133.00,6099,CA,County,550660,13266.99,205.75,205.75,13266.99,5,9,479.00,4.00
70655,2021-08-27,San Joaquin,California,89018,1513.00,6077,CA,County,762148,11679.88,198.52,198.52,11679.88,6,16,427.00,3.00
41067,2021-08-27,Riverside,California,340526,4727.00,6065,CA,County,2470546,13783.43,191.33,191.33,13783.43,7,7,938.00,8.00
110274,2021-08-27,Tulare,California,65554,870.00,6107,CA,County,466195,14061.50,186.62,186.62,14061.50,8,5,280.00,3.00
637713,2021-08-27,Merced,California,36352,506.00,6047,CA,County,277680,13091.33,182.22,182.22,13091.33,9,10,239.00,0.00
39989,2021-08-27,Fresno,California,124095,1770.00,6019,CA,County,999101,12420.67,177.16,177.16,12420.67,10,13,470.00,9.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5272,08/23/21,Lassen,6122,27.00,20024.20,88.31,35,1,42.00,1.00
5273,08/24/21,Lassen,6122,27.00,20024.20,88.31,35,1,0.00,0.00
5274,08/25/21,Lassen,6122,27.00,20024.20,88.31,35,1,0.00,0.00
5275,08/26/21,Lassen,6167,27.00,20171.39,88.31,35,1,45.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5358,08/23/21,Fresno,122059,1759.00,12216.88,176.06,10,13,292.00,0.00
5359,08/24/21,Fresno,122447,1759.00,12255.72,176.06,10,13,388.00,0.00
5360,08/25/21,Fresno,123026,1761.00,12313.67,176.26,10,13,579.00,2.00
5361,08/26/21,Fresno,123625,1761.00,12373.62,176.26,10,13,599.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)